# Clean Stanford Meter Data
Code author: Sahar H. El Abbadi
Date started: 2022-03-11
Date last edited: 2022-03-28

## Data description:
Stanford metering data, cleaned according to data processing pipeline. Includes 30-s, 60-s, and 90-s average flow rates, percent methane, and whole gas measurements. Each measurement is provided for the three different timestamp metrics: Stanford timestamps, as observed from the field; operator reported timestamps; and FlightRadar timestamps.

## Script objective:
- Load gas flow data for each flight day and assign methane mole fraction based on KM data
- Make "operator_meter" datasets for each method of using timestamp. Use whole-gas measurements from cleaned meter data, assign appropriate methane mole fraction, and calculate average and standard deviation of the release rate in kgh CH4 for each airplane overpass

In [1]:
import pandas as pd

# Setup

# Imports
from methods_source import load_summary_files, make_operator_meter_dataset, generate_daily_releases, load_flight_days


### Load daily metered release rates
This is all the metered data for releases conducted on flight days for each airplane operator. Includes the metered flow rate, then assigns methane mole fraction based on the measurement datetime, and calculates the methane mole fraction using the Kinder Morgan methane mole fraction data.

WARNING: this script takes a long time to run! Only run if you need to recreate the .csv files

In [2]:
# Comment out to prevent from accidentally running
load_flight_days()
generate_daily_releases()


Operator: Carbon Mapper
Flight Day: 10_10
Assigning km methane mole fraction for Carbon Mapper on 10_10
Assigning su_raw methane mole fraction for Carbon Mapper on 10_10
Assigning su_normalized methane mole fraction for Carbon Mapper on 10_10
Operator: Carbon Mapper
Flight Day: 10_11
Assigning km methane mole fraction for Carbon Mapper on 10_11
Assigning su_raw methane mole fraction for Carbon Mapper on 10_11
Assigning su_normalized methane mole fraction for Carbon Mapper on 10_11
Operator: Carbon Mapper
Flight Day: 10_12
Assigning km methane mole fraction for Carbon Mapper on 10_12
Assigning su_raw methane mole fraction for Carbon Mapper on 10_12
Assigning su_normalized methane mole fraction for Carbon Mapper on 10_12
Operator: Carbon Mapper
Flight Day: 10_28
Assigning km methane mole fraction for Carbon Mapper on 10_28
Assigning su_raw methane mole fraction for Carbon Mapper on 10_28
Assigning su_normalized methane mole fraction for Carbon Mapper on 10_28
Operator: Carbon Mapper
Flig

### Make operator meter files
Operator meter files (saved in 02_meter_data > operator_meter_data) are meter files that used as input for creating the operator overpass summary dataframe (03_results > overpass_summary), which in turn is the input for all major analysis of airplane performance.

Operator meter files have the following columns:
- datetime_utc: datetime in UTC timezone, dtype is datetime object
- overpass_id: matches PerformerExperimentID in the operator report
- phase_iii: indicates if we provided the operator with the flow rate for this overpass in Phase III, with 1 meaning we gave the overpass and 0 meaning we did not
- kgh_gas_30: average whole gas flow rate for the 30 seconds preceding the overpass time
- kgh_gas_60: average whole gas flow rate for the 60 seconds preceding the overpass time
- kgh_gas_90: average whole gas flow rate for the 90 seconds preceding the overpass time
- meter: indicates which meter was used for measurement (bc = small Coriolis meter, mc = medium Coriolis meter, pc = large Coriolis meter)
- qc_su_discard: 0 if Stanford did NOT discard, 1 if Stanford QC does indicate this should be discarded, using lax criteria. Lax criteria is the standard for all analysis, we compare to strict in 07_qc_comparison.ipynb
- qc_su_discard_strict: indicates if Stanford would discard (1) with strict criteria, or keep (0) with strict criteria
- altitude_feet: flight altitude in feet, averaged for 60 seconds prior to the plane being directly over the release point. (Note: units in 02_meter_data > summary_files are tracking data, and are in the units reported by the tracking system. This is feet for FlightTracker and meters for Kairos' reporting system, both in units above sea level. The script clean_meter_column_names converts these to feet above ground level)
- methane_fraction_su_raw: methane fraction using raw values from Stanford samples as provided by Eurofins Air Toxins laboratory
- kgh_ch4_30_su_raw, kgh_ch4_60_su_raw, kgh_ch4_90_su_raw: calculated flow rate in kgh CH4 over 30, 60, and 90 seconds respectively, using methane_fraction_su_raw
- methane_fraction_su_normalized: methane fraction using Stanford samples analyzed by Eurofins Air Toxins laboratory, normalized such that the sum of all components included in the analysis adds to 100%.
- kgh_ch4_30_su_normalized, kgh_ch4_60_su_normalized, kgh_ch4_60_su_normalized: methane flow rate averaged over the relevant time period using methane_fraction_su_normalized
- methane_fraction_km: methane mole fraction provided by Kinder Morgan for gas directed to the filling points used in this analysis. We used the analysis conducted on the dates which we filled the CNG trucks, applied to all dates which those trucks were used for releases
- kgh_ch4_30_km, kgh_ch4_60_km, kgh_ch4_90_km: methane flow rate averaged over the relevant time period, using methane_fraction_km

#### Scientific Aviation:
Note that Scientific Aviation submitted a report using non-standard template. The SciAv.csv summary file and sciav_1_overpasses overpass summary file were manually generated by Sahar El Abbadi.

In [ ]:
# Load data
cm_meter_raw, ghg_meter_raw, kairos_meter_raw, mair_meter_raw, sciav_meter_raw = load_summary_files()

timekeepers = ['flightradar', 'stanford', 'team']
operators = ['Carbon Mapper', 'GHGSat', 'Kairos', 'MethaneAir']
time_ave = [30, 60, 90]
gas_source = ['km', 'su_raw', 'su_normalized']
# Dictionary of raw meter data
meter_raw = {
    'Carbon Mapper': cm_meter_raw,
    'GHGSat': ghg_meter_raw,
    'Kairos': kairos_meter_raw,
    'MethaneAir': mair_meter_raw,
    'Scientific Aviation': sciav_meter_raw,
}

# Generate all datasets and save them
for timekeeper in timekeepers:
    for operator in operators:
        if operator == 'Scientific Aviation':
            pass # Not applicable for SciAv
        else:
            for time in time_ave:
                for source in gas_source:
                    print(f'Making {timekeeper} meter dataset for {operator} with {source} averaged over {time}')
                    make_operator_meter_dataset(operator, meter_raw[operator], timekeeper, time, source)

# cm_meter = make_operator_meter_dataset('Carbon Mapper', cm_meter_raw, 'flightradar', 60, 'km')
# ghg_meter = make_operator_meter_dataset('GHGSat', ghg_meter_raw, 'flightradar', 60, 'km')
# kairos_meter = make_operator_meter_dataset('Kairos', kairos_meter_raw, 'flightradar', 60, 'km')
# mair_meter = make_operator_meter_dataset('Methane Air', mair_meter_raw, 'flightradar', 60, 'km')

